In [1]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')

from sublimpy import turbulence
import matplotlib.pyplot as plt
from sublimpy import tidy
import metpy.constants

# Open data

## Open SOS Measurement Dataset

In [2]:
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df_5Min = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')
# convert time column to datetime
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df_5Min = tidy_df_5Min.set_index('time').sort_index().loc[start_date:end_date].reset_index()

output_fn = f'tidy_df_30Min_{start_date}_{end_date}_planar_fit.parquet'

# Open z0 datassets - created with `calculate_roughness_length.ipynb`

In [3]:
z0_andreas_df = pd.read_parquet("z0estimates/z0_andreas_df.parquet")
z0_andreas_df_weekly = pd.read_parquet("z0estimates/z0_andreas_df_weekly.parquet")

z0_df = pd.read_parquet("z0estimates/z0_df.parquet").reset_index()
z0_df_weekly = pd.read_parquet("z0estimates/z0_df_weekly.parquet")

# Create 30min dataset

In [4]:
tidy_df_30Min = tidy_df_5Min.set_index('time').groupby([
    pd.Grouper(freq='30Min'), 'tower', 'height', 'measurement', 'variable'
])['value'].mean().reset_index()

In [5]:
time_series = tidy_df_30Min.query("variable == 'u*_3m_c'").time

z0_andreas_df = z0_andreas_df.set_index('time').reindex(time_series, method ='bfill')
z0_andreas_df_weekly = z0_andreas_df_weekly.set_index('time').reindex(time_series, method ='bfill')
z0_df_weekly = z0_df_weekly.set_index('time').reindex(time_series, method ='bfill').interpolate(method='ffill')

/tmp/ipykernel_32307/4230238068.py:5: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  z0_df_weekly = z0_df_weekly.set_index('time').reindex(time_series, method ='bfill').interpolate(method='ffill')


In [6]:
tidy_df_30Min = tidy.tidy_df_add_variable(
    tidy_df_30Min,
    z0_andreas_df.z0,
    variable='z0_andreas',
    measurement='z0',
    height=3,
    tower='c'
)

tidy_df_30Min = tidy.tidy_df_add_variable(
    tidy_df_30Min,
    z0_andreas_df_weekly.z0,
    variable='z0_andreas_weekly',
    measurement='z0',
    height=3,
    tower='c'
)

tidy_df_30Min = tidy.tidy_df_add_variable(
    tidy_df_30Min,
    z0_df_weekly.z0_3m_c,
    variable='z0_windprofile_weekly',
    measurement='z0',
    height=3,
    tower='c'
)

In [7]:
src = tidy_df_30Min.query("measurement == 'z0'")
alt.Chart(src).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").scale(type='log'),
    alt.Color('variable:O')
).properties(width=1000)

alt.Chart(...)

In [8]:
tidy_df_30Min

,time,tower,height,measurement,variable,value
0,2022-11-30 00:00:00,c,0.0,IDir,IDir_c,139.000000
1,2022-11-30 00:00:00,c,0.0,SWE,SWE_p2_c,24.134980
2,2022-11-30 00:00:00,c,0.0,Vpile,Vpile_c,-0.000011
3,2022-11-30 00:00:00,c,0.0,Vtherm,Vtherm_c,2.004840
4,2022-11-30 00:00:00,c,0.0,air density,Tsurfairdensity_c,0.992988
...,...,...,...,...,...,...
2023-05-09 15:30:00,2023-05-09 15:30:00,c,3.0,z0,z0_windprofile_weekly,0.000844
2023-05-09 16:00:00,2023-05-09 16:00:00,c,3.0,z0,z0_windprofile_weekly,0.000844
2023-05-09 16:30:00,2023-05-09 16:30:00,c,3.0,z0,z0_windprofile_weekly,0.000844
2023-05-09 17:00:00,2023-05-09 17:00:00,c,3.0,z0,z0_windprofile_weekly,0.000844


In [9]:
output_fn

'tidy_df_30Min_20221130_20230509_planar_fit.parquet'

In [10]:
tidy_df_30Min.to_parquet(output_fn, index=False)